In [5]:
import pandas as pd
import numpy as np
from alive_progress import alive_bar, alive_it 

In [6]:
rankings_df = pd.read_csv("team_rankings.csv")
rankings_df.head()
# rankings_df["date"].info()

,date,url,1_team_name,1_team_id,1_team_ranking_points,2_team_name,2_team_id,2_team_ranking_points,3_team_name,3_team_id,...,27_team_ranking_points,28_team_name,28_team_id,28_team_ranking_points,29_team_name,29_team_id,29_team_ranking_points,30_team_name,30_team_id,30_team_ranking_points
0,2022-03-07,https://www.hltv.org/ranking/teams/2022/march/07,natus-vincere,4608,1000,gambit,6651,744,g2,5995,...,56,endpoint,7234,55,spirit,7020,51,eternal-fire,11251,50
1,2022-03-08,https://www.hltv.org/ranking/teams/2022/march/07,natus-vincere,4608,1000,gambit,6651,744,g2,5995,...,56,endpoint,7234,55,spirit,7020,51,eternal-fire,11251,50
2,2022-03-09,https://www.hltv.org/ranking/teams/2022/march/07,natus-vincere,4608,1000,gambit,6651,744,g2,5995,...,56,endpoint,7234,55,spirit,7020,51,eternal-fire,11251,50
3,2022-03-10,https://www.hltv.org/ranking/teams/2022/march/07,natus-vincere,4608,1000,gambit,6651,744,g2,5995,...,56,endpoint,7234,55,spirit,7020,51,eternal-fire,11251,50
4,2022-02-28,https://www.hltv.org/ranking/teams/2022/februa...,natus-vincere,4608,1000,gambit,6651,744,g2,5995,...,56,apeks,9806,50,skade,10386,49,bluejays,5347,48


In [7]:
matches_df = pd.read_csv("matches_stats.csv")
matches_df.head()
# matches_df.info()

C:\Users\alecw\AppData\Local\Temp\ipykernel_22788\3090517449.py:1: DtypeWarning: Columns (20,53,128) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_df = pd.read_csv("matches_stats.csv")


,match_url,match_date,map,t1_name,t1_id,t1_total_rw,t1_fh_rw,t1_sh_rw,t1_ot_rw,t1_rating,...,t2p5_kills,t2p5_hskills,t2p5_assists,t2p5_fassists,t2p5_deaths,t2p5_kdratio,t2p5_kddiff,t2p5_adr,t2p5_fkdiff,t2p5_rating
0,/stats/matches/mapstatsid/46584/hellraisers-vs...,2017-05-17,train,hellraisers,5310,16,13,3,0,1.66,...,5,2,1,0.0,17,33.3%,-12,35.9,-3,0.26
1,/stats/matches/mapstatsid/62534/envy-vs-north?...,2018-03-08,cobblestone,north,7533,16,8,8,0,1.02,...,15,9,1,0.0,22,63.3%,-7,58.5,-3,0.78
2,/stats/matches/mapstatsid/61361/fnatic-vs-gamb...,2018-02-17,inferno,fnatic,4991,16,10,6,0,1.28,...,7,5,3,1.0,22,45.8%,-15,49.2,-3,0.37
3,/stats/matches/mapstatsid/57159/liquid-vs-clou...,2017-11-12,overpass,liquid,5973,5,3,2,0,0.89,...,16,5,3,0.0,16,71.4%,0,67.6,0,1.00
4,/stats/matches/mapstatsid/123194/dbl-poney-vs-...,2021-07-04,ancient,dbl-poney,11003,16,6,10,0,1.04,...,15,10,2,0.0,22,56.7%,-7,64.6,-1,0.81


<class 'pandas.core.series.Series'>
RangeIndex: 15098 entries, 0 to 15097
Series name: t1p1_fassists
Non-Null Count  Dtype  
--------------  -----  
14017 non-null  float64
dtypes: float64(1)
memory usage: 118.1 KB


In [8]:
# now lets add ranking team ranking information to the matches dataset #

# extract the two team name columns
t1_names = matches_df["t1_name"]
t2_names = matches_df["t2_name"]

t1_rankings = []
t2_rankings = []
# with alive_bar(len(matches_df), force_tty=True):
for i in range(len(matches_df)):
    date = matches_df["match_date"].iloc[i]
    rankings_series = rankings_df.loc[rankings_df["date"] == date, :]
    t1_rank = int(x.index[0].split("_")[0]) if (x:=rankings_series[rankings_series == t1_names[i]]).size == 1 else 0
    t2_rank = int(x.index[0].split("_")[0]) if (x:=rankings_series[rankings_series == t2_names[i]]).size == 1 else 0
    t1_rankings.append(t1_rank)
    t2_rankings.append(t2_rank)
        # bar()
        
t1_rankings = pd.Series(t1_rankings).replace(0, np.nan)
t2_rankings = pd.Series(t2_rankings).replace(0, np.nan)


matches_dataset = matches_df.copy()


t1_rank_col_index = matches_df.columns.get_loc("t1_id") + 1
matches_dataset.insert(t1_rank_col_index, "t1_rank", t1_rankings, allow_duplicates=False)

# get updated index now that column has been inserted prior
t2_rank_col_index = matches_df.columns.get_loc("t2_id") + 1
matches_dataset.insert(t2_rank_col_index, "t2_rank", t2_rankings, allow_duplicates=False)

print(x:=matches_dataset.tail().iloc[:5, t1_rank_col_index - 1: t1_rank_col_index + 5])
print(y:=matches_dataset.tail().iloc[:5, t2_rank_col_index - 1: t2_rank_col_index + 5])

print(matches_dataset.info())

# force team names to be lowercase
matches_dataset["t1_name"] = matches_dataset["t1_name"].str.lower()
matches_dataset["t2_name"] = matches_dataset["t2_name"].str.lower()

matches_dataset.to_csv(index=False, path_or_buf="matches_dataset.csv")

       t1_id  t1_rank  t1_total_rw  t1_fh_rw  t1_sh_rw  t1_ot_rw
15093  11595      NaN            7         6         1         0
15094   6667      NaN           16        12         4         0
15095  11595      NaN           17         9         6         2
15096  11595      NaN           22         9         6         7
15097   5995      NaN            2         2         0         0
      t2_name  t2_rank  t2_id  t2_total_rw  t2_fh_rw  t2_sh_rw
15093   furia      NaN   8297           16         9         7
15094  sprout      NaN   8637            9         3         6
15095    ence      NaN   4869           19         6         9
15096    faze      NaN   6667           18         6         9
15097  fnatic      NaN   4991           16        13         3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Columns: 133 entries, match_url to t2p5_rating
dtypes: float64(31), int64(74), object(28)
memory usage: 15.3+ MB
None


In [9]:
"""
import csv

def format_date(hltv_date : str):
    month, day, year = hltv_date.split("/")
    return "-".join((f"20{year}", month.rjust(2,'0'), day.rjust(2,'0')))

output = []
fieldnames = []

with open("matches_stats.csv","r", encoding="utf-8") as f:
    csvreader = csv.DictReader(f)
    fieldnames = list(dict(list(csvreader)[0]).keys())

with open("matches_stats.csv","r", encoding="utf-8") as f:
    csvreader = csv.DictReader(f)
    # fieldnames = list(dict(list(csvreader)[0]).keys())
    for row in csvreader:
        # row["match_date"] = format_date(row["match_date"])
        output.append(row)
print(len(output))

with open("matches_stats.csv","w", encoding="utf-8", newline="") as f:
    csvwriter = csv.DictWriter(f, fieldnames=fieldnames)
    csvwriter.writeheader()
    csvwriter.writerows(output)

"""

'\nimport csv\n\ndef format_date(hltv_date : str):\n    month, day, year = hltv_date.split("/")\n    return "-".join((f"20{year}", month.rjust(2,\'0\'), day.rjust(2,\'0\')))\n\noutput = []\nfieldnames = []\n\nwith open("matches_stats.csv","r", encoding="utf-8") as f:\n    csvreader = csv.DictReader(f)\n    fieldnames = list(dict(list(csvreader)[0]).keys())\n\nwith open("matches_stats.csv","r", encoding="utf-8") as f:\n    csvreader = csv.DictReader(f)\n    # fieldnames = list(dict(list(csvreader)[0]).keys())\n    for row in csvreader:\n        # row["match_date"] = format_date(row["match_date"])\n        output.append(row)\nprint(len(output))\n\nwith open("matches_stats.csv","w", encoding="utf-8", newline="") as f:\n    csvwriter = csv.DictWriter(f, fieldnames=fieldnames)\n    csvwriter.writeheader()\n    csvwriter.writerows(output)\n\n'

In [10]:
def get_matches_by_team(team_name : str):
    return matches_stats_df.loc[ ((matches_stats_df["t1_name"] == team_name.lower()) 
                                | (matches_stats_df["t2_name"] == team_name.lower())) , :]

g2_matches_df = get_matches_by_team("G2")
faze_matches_df = get_matches_by_team("FaZe")

print(g2_matches_df.info())
print("")
print(faze_matches_df.info())

g2_matches_df[""].value_counts()


NameError: name 'matches_stats_df' is not defined